# Monte Carlo approaches to prediction and control

In this notebook, you will implement the Monte Carlo approaches to prediction and control described in [Sutton and Barto's book, Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html). We will use the grid ```World``` class from the previous lecture, but now without relying on knowledge of the task dynamics, that is, without relying on knowledge about transition probabilities.

### Install dependencies

In [1]:
! pip install numpy pandas

### Imports

In [2]:
import numpy as np
import random
import sys          # We use sys to get the max value of a float
import pandas as pd # We only use pandas for displaying tables nicely
pd.options.display.float_format = '{:,.3f}'.format

### ```World``` class and globals

The ```World``` is a grid represented as a two-dimensional array of characters where each character can represent free space, an obstacle, or a terminal. Each non-obstacle cell is associated with a reward that an agent gets for moving to that cell (can be 0). The size of the world is _width_ $\times$ _height_ characters.

A _state_ is a tuple $(x,y)$.

An empty world is created in the ```__init__``` method. Obstacles, rewards and terminals can then be added with ```add_obstacle``` and ```add_reward```.

To calculate the next state of an agent (that is, an agent is in some state $s = (x,y)$ and performs and action, $a$), ```get_next_state()```should be called.

__Note that ```get_state_transition_probabilities``` has been removed and an agent must now rely on experience interacting with a world to learn.__

In [3]:
# Globals:
ACTIONS = ("up", "down", "left", "right")

# Rewards, terminals and obstacles are characters:
REWARDS = {" ": 0, ".": 0.1, "+": 10, "-": -10}
TERMINALS = ("+", "-") # Note a terminal should also have a reward assigned
OBSTACLES = ("#")

# Discount factor
gamma = 1

# The probability of a random move:
rand_move_probability = 0

class World:
  def __init__(self, width, height):
    self.width = width
    self.height = height
    # Create an empty world where the agent can move to all cells
    self.grid = np.full((width, height), ' ', dtype='U1')

  def add_obstacle(self, start_x, start_y, end_x=None, end_y=None):
    """
    Create an obstacle in either a single cell or rectangle.
    """
    if end_x == None: end_x = start_x
    if end_y == None: end_y = start_y

    self.grid[start_x:end_x + 1, start_y:end_y + 1] = OBSTACLES[0]

  def add_reward(self, x, y, reward):
    assert reward in REWARDS, f"{reward} not in {REWARDS}"
    self.grid[x, y] = reward

  def add_terminal(self, x, y, terminal):
    assert terminal in TERMINALS, f"{terminal} not in {TERMINALS}"
    self.grid[x, y] = terminal

  def is_obstacle(self, x, y):
    if x < 0 or x >= self.width or y < 0 or y >= self.height:
      return True
    else:
      return self.grid[x ,y] in OBSTACLES

  def is_terminal(self, x, y):
    return self.grid[x ,y] in TERMINALS

  def get_reward(self, x, y):
    """
    Return the reward associated with a given location
    """
    return REWARDS[self.grid[x, y]]

  def get_next_state(self, current_state, action):
    """
    Get the next state given a current state and an action. The outcome can be
    stochastic  where rand_move_probability determines the probability of
    ignoring the action and performing a random move.
    """
    assert action in ACTIONS, f"Unknown acion {action} must be one of {ACTIONS}"

    x, y = current_state

    # If our current state is a terminal, there is no next state
    if self.grid[x, y] in TERMINALS:
      return None

    # Check of a random action should be performed:
    if np.random.rand() < rand_move_probability:
      action = np.random.choice(ACTIONS)

    if action == "up":      y -= 1
    elif action == "down":  y += 1
    elif action == "left":  x -= 1
    elif action == "right": x += 1

    # If the next state is an obstacle, stay in the current state
    return (x, y) if not self.is_obstacle(x, y) else current_state


## Basic example: Generating episodes

An episode is the series of states, actions and rewards reflecting an agent's experience interacting with the environment. An episode starts with an agent being placed at some initial state and continues till the agent reaches a terminal state.  To generate episodes, we first need a world and a policy:


In [4]:
world = World(2, 3)

# Since we only focus on episodic tasks, we must have a terminal state that the
# agent eventually reaches
world.add_terminal(1, 2, "+")

def equiprobable_random_policy(x, y):
  return { k:1/len(ACTIONS) for k in ACTIONS }

print(equiprobable_random_policy(1,1))

print(world.grid.T)

{'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
[[' ' ' ']
 [' ' ' ']
 [' ' '+']]


To generate an episode, we need to provide a ```World```, a policy, and a start state.

In each step, we do the following:
1. perform one of the actions (weighted random) returned by the policy for the giving state
2. get the reward and add a new entry to the episode $[S_t, A_t, R_{t+1}]$
3. move the agent to the next state

When a terminal state is reached, we return all the $[[S_0, A_0, R_1], ..., [S_{T}, A_T, R_{T+1}]]$ observed in the episode.

In [5]:
def generate_episode(world, policy, start_state):
    current_state = start_state
    episode = []
    while not world.is_terminal(*current_state):
        # Get the possible actions and their probabilities that our policy says
        # that the agent should perform in the current state:
        possible_actions = policy(*current_state)

        # Pick a weighted random action:
        action = random.choices(population=list(possible_actions.keys()),
                                weights=possible_actions.values(), k=1)

        # Get the next state from the world
        next_state = world.get_next_state(current_state, action[0])

        # Get the reward for performing the action
        reward = world.get_reward(*next_state)

        # Save the state, action and reward for this time step in our episode
        episode.append([current_state, action[0], reward])

        # Move the agent to the new state
        current_state = next_state

    return episode

Now, we can try to generate a couple of episodes and print the result:

In [6]:
for i in range(5):
    print(f"Episode {i}:")
    episode = generate_episode(world, equiprobable_random_policy, (0, 0))
    print(pd.DataFrame(episode, columns=["State", "Action", "Reward"]), end="\n\n")

Episode 0:
    State Action  Reward
0  (0, 0)  right       0
1  (1, 0)     up       0
2  (1, 0)   down       0
3  (1, 1)   down      10

Episode 1:
     State Action  Reward
0   (0, 0)   left       0
1   (0, 0)     up       0
2   (0, 0)  right       0
3   (1, 0)   down       0
4   (1, 1)  right       0
5   (1, 1)   left       0
6   (0, 1)   down       0
7   (0, 2)     up       0
8   (0, 1)     up       0
9   (0, 0)   left       0
10  (0, 0)   down       0
11  (0, 1)  right       0
12  (1, 1)     up       0
13  (1, 0)   left       0
14  (0, 0)   down       0
15  (0, 1)     up       0
16  (0, 0)  right       0
17  (1, 0)   left       0
18  (0, 0)  right       0
19  (1, 0)     up       0
20  (1, 0)     up       0
21  (1, 0)   down       0
22  (1, 1)   down      10

Episode 2:
     State Action  Reward
0   (0, 0)  right       0
1   (1, 0)   down       0
2   (1, 1)   left       0
3   (0, 1)     up       0
4   (0, 0)  right       0
5   (1, 0)  right       0
6   (1, 0)   down       0
7   (1, 

### Exercise: Implement Monte Carlo-based prediction for state values

You should implement first-visit MC prediction for estimating $V≈v_\pi$. See page 92 of [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html).


In [7]:
### TODO: Implement your code here
def first_visit_MC(world, policy):
    V = np.full((world.width, world.height), 0.0)
    sum_of_returns = np.full((world.width, world.height), 0.0)
    times_visited = np.full((world.width, world.height), 0.0)


    for _ in range(5000):
        episode = generate_episode(world, policy, (np.random.randint(world.width-1), np.random.randint(world.height-1)))

        G = 0

        # loop for each step in episode
        for i in reversed(range(len(episode))): #loop t=T-1, T-2,..,0
            state, action, reward = episode[i]
            G = gamma * G + reward
            isVisited = False
            for j in range(i-1):    # Tjek om det first visit
                previous_state = episode[j][0]
                if state == previous_state: 
                    isVisited = True
                    break

            if not isVisited:       # Ellers "append G" og returner avg G
                sum_of_returns[state] += G
                times_visited[state] += 1
                V[state] = sum_of_returns[state] / times_visited[state]
        

    return V

First, try your algorithm on the small $2\times3$ world above using an equiprobable policy and $\gamma = 0.9$. Depending on the number of episodes you use, you should get close to the true values:

<table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>3.283</td>
      <td>3.616</td>
    </tr>
    <tr>
      <th>1</th>
      <td>4.409</td>
      <td>5.556</td>
    </tr>
    <tr>
      <th>2</th>
      <td>6.349</td>
      <td>0.000</td>
    </tr>
  </tbody>
</table>




In [8]:
gamma = 0.9

### TODO: Implement your code here
V1 = first_visit_MC(world, equiprobable_random_policy)

print(pd.DataFrame(V1).T)

      0     1
0 3.225 3.552
1 4.408 5.529
2 6.289 0.000


Try to run your MC prediction code on worlds of different sizes (be careful not to make your world too large or you should have multiple terminals that an agent is likely to hit, otherwise it may take too long). You can try to change the policy as well, but rememeber that the agent **must** eventually reach a terminal state under any policy that you try.

In [9]:
### TODO: Implement your code here
world2 = World(4, 4)

# Since we only focus on episodic tasks, we must have a terminal state that the
# agent eventually reaches
world2.add_terminal(1, 2, "+")
world2.add_terminal(3, 3, "+")

print(world2.grid.T)

V2 = first_visit_MC(world2, equiprobable_random_policy)
print(pd.DataFrame(V2).T)

[[' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']
 [' ' '+' ' ' ' ']
 [' ' ' ' ' ' '+']]
      0     1     2     3
0 2.842 3.092 2.801 2.742
1 4.009 4.971 4.033 3.851
2 5.805 0.000 6.336 6.256
3 4.972 6.551 6.971 0.000


### Exercise: Implement Monte Carlo-based prediction for state-action values

There is one more step that has to be in place before we can start to optimize a policy: estimating state-action values, $q_\pi(s,a)$, based on experience. Above where we estimated $v_\pi$, we only needed to keep track of the average return observed for _each state_. However, in order to estimate state-action values, we need to compute the average return observed for _each state-action_ pair.

That is, for every state $(0,0), (0,1), (0,2)...$ we need to compute different estimates for the four actions ```[ "up", "down", "left", "right" ]```

In [10]:
### TODO: Implement your code here to predict state-action values.
def exploring_starts_MC(world):
    pol = np.full((world.width, world.height), None)
    Q = np.full((world.width, world.height, 4), 0.0)
    sum_of_returns = np.full((world.width, world.height, 4), 0.0)
    times_visited = np.full((world.width, world.height, 4), 0.0)

    action_to_index = {action: idx for idx, action in enumerate(ACTIONS)}

    for _ in range(5000):
        episode = generate_episode(world, equiprobable_random_policy, (np.random.randint(world.width-1), np.random.randint(world.height-1)))
        G = 0

        for i in reversed(range(len(episode))):
            G = gamma * G + episode[i][2]
            isVisited = False
            for j in range(i-1):
                if episode[i][0] == episode[j][0] and episode[i][1] == episode[j][1]:
                    isVisited = True
                    break

            if not isVisited:
                this_action = action_to_index[episode[i][1]]
                x, y = episode[i][0]
                sum_of_returns[x, y, this_action] += G
                times_visited[x, y, this_action] += 1
                Q[x, y, this_action] = sum_of_returns[x, y, this_action] / times_visited[x, y, this_action]
                
                pol[x, y] = ACTIONS[np.argmax(Q[x, y])]
        

    return pol       
    




Try to experiment with your implementation by running it on different world sizes (be careful not to make your world too large or you should have multiple terminals that an agent is likely to hit, otherwise it may take too long), and try to experiment with different numbers of episodes:

In [11]:
### TODO: Implement your code here
pol = exploring_starts_MC(world)
    

print(pol.T)


[['down' 'down']
 ['down' 'down']
 ['right' None]]


### Exercise: Implement on-policy Monte Carlo-based control with an $\epsilon$-soft policy

You are now ready to implement MC-based control (see page 101 of [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html) for the algorithm).

In your implementation, you need to update the state-action estimates like in the exercise above, but now, you also need implement an $ϵ$-soft policy that you can modify. How could you do that?

_Hint_: You can either represent your policy explicitly. That is, for each state $(x,y)$ you have a ```dict``` with actions and their probabilities which you then update each time you step through an episode. When the policy is called, it then just returns the ```dict``` with action probablities corresponding to the current state.

Alternatively, you can compute the action probabilities when your policy is called based on the current action-values estimates.

In [21]:
gamma = 0.9
epsilon = 0.1

sum_of_returns_soft = np.full((world.width, world.height, 4), 0.0)
times_visited_soft = np.full((world.width, world.height, 4), 1.0)

def MC_soft_policy(x,y):
    Q = []
    for acts in ACTIONS:
        Q.append(sum_of_returns_soft[x,y,ACTIONS.index(acts)] / times_visited_soft[x,y,ACTIONS.index(acts)])
    
    best_action = ACTIONS[np.argmax(Q)]
    # print(best_action)

    if np.random.rand() < 1 - epsilon:
        return {best_action:1}
    else:
        return { k:1/len(ACTIONS) for k in ACTIONS }
        

MC_soft_policy(1,1)

def reset_soft(world):
    global sum_of_returns_soft, times_visited_soft
    sum_of_returns_soft = np.full((world.width, world.height, 4), 0.0)
    times_visited_soft = np.full((world.width, world.height, 4), 1.0)


### TODO: Implement you code here. You need to define a policy function
###       and then the actual algorithm that goes through time step in each
###       episode and updates state-action values and the policy. Also, make
###       sure that you can print out the policy learned, that is, the action
###       with the highest expected value in each state.
def soft_MC(world):
    pol = np.full((world.width, world.height), None)
    Q = np.full((world.width, world.height, 4), 0.0)

    action_to_index = {action: idx for idx, action in enumerate(ACTIONS)}

    for _ in range(5000):
        episode = generate_episode(world, MC_soft_policy, (np.random.randint(world.width-1), np.random.randint(world.height-1)))
        G = 0

        for i in reversed(range(len(episode))):
            state, action, reward = episode[i]
            G = gamma * G + reward
            isVisited = False
            for j in range(i-1):
                previous_state, previous_action, previous_reward = episode[j]
                if state == previous_state and action == previous_action:
                    isVisited = True
                    break

            if not isVisited:
                this_action = action_to_index[action]
                x, y = state
                sum_of_returns_soft[x, y, this_action] += G
                times_visited_soft[x, y, this_action] += 1
                Q[x, y, this_action] = sum_of_returns_soft[x, y, this_action] / times_visited_soft[x, y, this_action]
                
                pol[x, y] = ACTIONS[np.argmax(Q[x, y])]
        

    return pol      

Try to experiment with your implementation by running it on different world sizes (be careful not to make your world too large or you should have multiple terminals that an agent is likely to hit, otherwise it may take too long), try to experiment with different numbers of episodes, and different values of epsilon:

In [23]:
### TODO: Implement your code here
epsilon = 0.1

pol1 = soft_MC(world)

#print(pol1.T)
print("epsilon = 0.1")
display(pd.DataFrame(world.grid.T))
display(pd.DataFrame(pol1.T))

reset_soft(world2)
pol2 = soft_MC(world2)

#print(pol2.T)
print("epsilon = 0.1")
display(pd.DataFrame(world2.grid.T))
display(pd.DataFrame(pol2.T))

reset_soft(world2)
epsilon = 0.3 # Den her finder oftere den bedste løsning (ses ved 3.kolonne = ned og (2,3) = right)
pol3 = soft_MC(world2)
#print(pol3.T)
print("epsilon = 0.3")
display(pd.DataFrame(world2.grid.T))
display(pd.DataFrame(pol3.T))



epsilon = 0.1


,0,1
0,,
1,,
2,,+


,0,1
0,down,down
1,right,down
2,right,None


epsilon = 0.1


,0,1,2,3
0,,,,
1,,,,
2,,+,,
3,,,,+


,0,1,2,3
0,right,down,down,down
1,right,down,down,left
2,right,None,left,up
3,up,up,up,None


epsilon = 0.3


,0,1,2,3
0,,,,
1,,,,
2,,+,,
3,,,,+


,0,1,2,3
0,down,down,down,down
1,right,down,left,down
2,right,None,left,down
3,up,up,right,None


### Optional exercise

Try to implement exploring starts (see page 99 of [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html) for the algorithm). It should be straightforward and only require minimal changes to the code for the exercise above.

### JEG IKKE SIKKER PÅ AT JEG HAR LAVET DET OPGAVEN FAKTISK SPURGTE OM

In [14]:
sum_of_returns_soft_exp = np.full((world.width, world.height, 4), 0.0)
times_visited_soft_exp = np.full((world.width, world.height, 4), 1.0)
isStart = True

def MC_soft_exploring_policy(x,y):
    global isStart
    Q = []
    for acts in ACTIONS:
        Q.append(sum_of_returns_soft_exp[x,y,ACTIONS.index(acts)] / times_visited_soft_exp[x,y,ACTIONS.index(acts)])
    
    best_action = ACTIONS[np.argmax(Q)]
    # print(best_action)
    if isStart:
        return { k:1/len(ACTIONS) for k in ACTIONS }

    if np.random.rand() < 1 - epsilon or not isStart:
        return {best_action:1}
    else:
        isStart = False
        return { k:1/len(ACTIONS) for k in ACTIONS }


def reset_soft_exploring(world):
    global sum_of_returns_soft_exp, times_visited_soft_exp, isStart
    sum_of_returns_soft_exp = np.full((world.width, world.height, 4), 0.0)
    times_visited_soft_exp = np.full((world.width, world.height, 4), 1.0)
    isStart = True

def soft_exploring_MC(world):
    global isStart
    pol = np.full((world.width, world.height), None)
    Q = np.full((world.width, world.height, 4), 0.0)

    action_to_index = {action: idx for idx, action in enumerate(ACTIONS)}

    for _ in range(5000):
        
        episode = generate_episode(world, MC_soft_exploring_policy, (np.random.randint(world.width-1), np.random.randint(world.height-1)))
        G = 0

        for i in reversed(range(len(episode))):
            G = gamma * G + episode[i][2]
            isVisited = False
            for j in range(i-1):
                if episode[i][0] == episode[j][0] and episode[i][1] == episode[j][1]:
                    isVisited = True
                    break

            if not isVisited:
                this_action = action_to_index[episode[i][1]]
                x, y = episode[i][0]
                sum_of_returns_soft_exp[x, y, this_action] += G
                times_visited_soft_exp[x, y, this_action] += 1
                Q[x, y, this_action] = sum_of_returns_soft_exp[x, y, this_action] / times_visited_soft_exp[x, y, this_action]
                
                pol[x, y] = ACTIONS[np.argmax(Q[x, y])]
        
        isStart = True
        

    return pol      

In [15]:
epsilon = 0.1

# Selvom antal episoder er sat ned til 5000 (fordi den her åbenbart tager meget længere tid), 
# så finder den stadig nogle gode policys fordi den er mere tilbøjelig til at udforske på første forsøg 
# fx så var (2,3) med eps = 0.1 ikke altid udforsket nok til at vide der var en terminal state lige til højre for den.
# Men her hvor den altid udforsker første action så er der en 1/64 del chance for at den finder netop den sti. 
#      16 felter og 4 actions => 1/16 * 1/4 = 1/64 del chance for lige netop det felt og action

reset_soft_exploring(world2)
pol4 = soft_exploring_MC(world)

print(pol4.T)

reset_soft_exploring(world2)
pol5 = soft_exploring_MC(world2)

print(pol5.T)

reset_soft_exploring(world2)
epsilon = 0.3 
pol6 = soft_exploring_MC(world2)
print(pol6.T)

[['down' 'down']
 ['down' 'down']
 ['right' None]]
[['down' 'down' 'down' 'down']
 ['down' 'down' 'down' 'down']
 ['right' None 'left' 'down']
 ['right' 'up' 'right' None]]
[['down' 'down' 'down' 'down']
 ['down' 'down' 'down' 'down']
 ['right' None 'left' 'down']
 ['right' 'up' 'right' None]]
